In [1]:
#-----------------------------------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")
import time as t
from datetime import datetime, timedelta, date 
from dateutil.relativedelta import *
from netCDF4 import Dataset   
from glob import glob
import requests 
import pandas as pd
import xarray as xr
import requests 
import functools
import operator
import wget
import os
import gc
import numpy as np
import pygrib
import seaborn as sns
from sklearn.linear_model import LinearRegression
from scipy import signal
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.decomposition import PCA
from statsmodels.tsa.stattools import adfuller
import datetime
import holidays
import xarray as xr


import matplotlib.pyplot as plt
import geopandas as gpd
import shapely
import shapefile
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import cartopy, cartopy.crs as ccrs   
import regionmask
from shapely.geometry import Point, Polygon

from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as mticker 
import scipy.ndimage as ndimage
from PIL import Image


In [2]:
files= glob('/mnt/c/scripts/temas/stream_plot/dados_grib/psi200_01_*.grb2')
li = sorted(files)


In [3]:

data_list = []
# Loop 
for q in li:
   
    dataset = xr.open_dataset(q)
#     dataset = dataset.assign_coords({'number':1})
    dataset = dataset.swap_dims({"step": "valid_time"}).drop('step')  #transforma valid_time em dim e retira a coord step
    strf = dataset['strf'].expand_dims('time') #separar cada arquivo pelo star_time
    data_list.append(strf)


In [4]:

merge = data_list[0]  # Inicializa merge com o primeiro elemento da lista

for i in range(1, len(data_list)):
    merge = xr.merge([merge, data_list[i]], join='outer')

In [5]:
merge

<xarray.Dataset>
Dimensions:        (latitude: 181, longitude: 360, valid_time: 1252, time: 6)
Coordinates:
  * latitude       (latitude) float64 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * longitude      (longitude) float64 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * valid_time     (valid_time) datetime64[ns] 2023-05-24T06:00:00 ... 2024-0...
  * time           (time) datetime64[ns] 2023-05-24 2023-05-25 ... 2023-05-29
    isobaricInhPa  float64 200.0
Data variables:
    strf           (time, valid_time, latitude, longitude) float32 -1.298e+08...
Attributes: (12/29)
    GRIB_paramId:                             1
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      65160
    GRIB_typeOfLevel:                         isobaricInhPa
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                Stream function
    GRIB_shortName:                           strf
    GRIB_units:                               m**2 s**-1
    long_name:                                Stream function
    units:                                    m**2 s**-1
    standard_name:                            unknown

In [6]:
merge.mean('time')

<xarray.Dataset>
Dimensions:        (latitude: 181, longitude: 360, valid_time: 1252)
Coordinates:
  * latitude       (latitude) float64 90.0 89.0 88.0 87.0 ... -88.0 -89.0 -90.0
  * longitude      (longitude) float64 0.0 1.0 2.0 3.0 ... 357.0 358.0 359.0
  * valid_time     (valid_time) datetime64[ns] 2023-05-24T06:00:00 ... 2024-0...
    isobaricInhPa  float64 200.0
Data variables:
    strf           (valid_time, latitude, longitude) float32 -1.298e+08 ... 1...